In [1]:
import os
os.chdir('../')

In [2]:
import json

class MockAIGame:
    observations = []
    rewards = []
    env = None

    def __init__(self, env):
        self.env = env
    
config = None
with open('config.json') as f:
    config = json.load(f)


In [3]:
import gym
env = gym.make('CartPole-v1')

In [4]:
from neurosim.game_interface import GameInterface

game = MockAIGame(env)
gi = GameInterface(game, config)


In [5]:
import time

from neurosim.conf import read_conf
from neurosim.critic import Critic

dconf = read_conf('config.json')
critic = Critic(dconf)


steps = []
observations = []
firing_rates = []

for i_episode in range(3):
    observation = env.reset()
    print(observation)
    for t in range(20):
        env.render()
#         print(observation)
        observations.append(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        rrr = critic.calc_reward(observation, observations[-1])
        
        game.observations = [observation]
        firing_rates.append(gi.input_firing_rates())
        firing_indices = (firing_rates[-1] > 0).nonzero()
        
        print('R' if action else 'L', [round(k, 3) for k in observation], rrr)
        
        if done:
            observations.append(observation)
            steps.append(t+1)
#             print("Episode finished after {} timesteps".format(t+1))
            break
        time.sleep(4)

env.close()


[ 0.00426228 -0.00149451  0.00650002 -0.01660919]
R [0.004, 0.194, 0.006, -0.307] -1.0195585745290625
L [0.008, -0.002, 0.0, -0.013] 1.0420825559501499
R [0.008, 0.193, -0.0, -0.305] -1.034765043203992
R [0.012, 0.389, -0.006, -0.598] -1.035282454068159
R [0.02, 0.584, -0.018, -0.893] -1.0429095622909061


KeyboardInterrupt: 

In [9]:
cols = ['pos', 'velocity', 'angle', 'angular velocity']
min_values = [-0.530120, -2.154456, -0.209409, -2.590083]
max_values = [0.610815, 1.796593, 0.209186, 2.664974]

In [14]:
import numpy as np
STEPS = 200


observation = [0] * 4
for ang in np.linspace(min_values[2], max_values[2], STEPS):
    observation[2] = ang
    for angv in np.linspace(min_values[3], max_values[3], STEPS):
        observation[3] = angv
        
        

(200,)